In [49]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
import undetected_chromedriver as uc

In [2]:
def getAbbreviations():
    df_convert = pd.read_csv('teamAbbreviations.csv')
    df_convert['Url'] = [x.replace(' ', '-').lower().replace('.', '') for x in df_convert['Full Name']]
    return df_convert

In [3]:
df_convert = getAbbreviations()
df_convert.head()

,Full Name,Abbreviation,League,Mascot,City,Url
0,Atlanta Hawks,ATL,NBA,Hawks,Atlanta,atlanta-hawks
1,Boston Celtics,BOS,NBA,Celtics,Boston,boston-celtics
2,Brooklyn Nets,BKN,NBA,Nets,Brooklyn,brooklyn-nets
3,Charlotte Hornets,CHA,NBA,Hornets,Charlotte,charlotte-hornets
4,Chicago Bulls,CHI,NBA,Bulls,Chicago,chicago-bulls


In [4]:
def initDriver(url, driver):
    if driver is None: 
        options = Options()
        driver = uc.Chrome(options=options)
        
    driver.get(url)
    return driver

In [5]:
def findPlayer(driver, player, team, league):
    time.sleep(1)
    players = driver.find_elements(By.XPATH, '//a[@class="sportsbook-row-name__wrapper-link"]')
    links = [playerLink.get_attribute('href') for playerLink in players]

    for i in range(len(players)):
        playerFound = players[i].find_element(By.CLASS_NAME, 'sportsbook-row-name')
        if playerFound.text == player:
            driver.get(links[i])
            break
        
    return None

In [80]:
def scrapeLines(driver, player, team, league):
    time.sleep(5)
    categories = driver.find_elements(By.CLASS_NAME, "market__header")
    first = 0
    
    for category in categories:
        if (first == 0):
            first += 1
        else:
            category.click()
        driver.execute_script("window.scrollBy(0, 200);")
        time.sleep(0.5)
            
    lines = driver.find_elements(By.CLASS_NAME, "market__button-points")
    odds = driver.find_elements(By.CLASS_NAME, "market__button-odds")
    
    print("here")
    
    i = 0
    for category in categories:
        line = lines[int(i/2)].text
        over, under = odds[i].text, odds[i + 1].text
        i += 2
        
        print(line + " " + over + " " + under)
        
    
    return None
    """
    row = driver.find_element(By.CLASS_NAME, "Table__sub-header")
    categories = row.find_elements(By.CSS_SELECTOR, "th.Table__TH")
    row = driver.find_element(By.XPATH, './/tr[@data-idx="0"]')
    stats = row.find_elements(By.CSS_SELECTOR, "td.Table__TD")
    players = {'Name': player, 'Team': team, 'League': league, 'Date': date}
    
    if date.date() == datetime.strptime(stats[0].text[4:] + '/2023', '%m/%d/%Y').date():  
        for i in range(len(categories)):
            players[categories[i].text] = stats[i].text
            
    return players"""

In [81]:
def run(df_query):
    lineLst = []
    driver = None
    sportDict = {"MLB": "baseball", "NHL": "hockey", "NBA": "basketball", "NFL": "football"}
    
    try:
        for index, row in df_query.iterrows():
            player, team, league = row["Name"], row["Team"], row["League"]
            url = 'https://sportsbook.draftkings.com/teams/' + sportDict[league] + '/' + league.lower() + '/' + df_convert.loc[(df_convert['Abbreviation'] == team) & (df_convert['League'] == league), 'Url'].values[0]
            driver = initDriver(url, driver)
            findPlayer(driver, player, team, league)
            lineLst.append(scrapeLines(driver, player, team, league))
    finally:
        driver.quit()
            
    return pd.DataFrame(lineLst)

In [82]:
data = [['Matthew Boyd', 'DET', 'MLB'], ['Thairo Estrada', 'SF', 'MLB']]
df_query = pd.DataFrame(data, columns=['Name', 'Team', 'League'])
df = run(df_query)

here
5.5 +105 −140
5.5 −115 −115
15.5 +265 −390
15.5 +130 −170
5.5 −140 +105
5.5 −105 −125
here


In [40]:
dfCopy = df.copy()

In [9]:
df = dfCopy.copy()

In [10]:
df.shape

(2, 34)

In [11]:
df.columns

Index(['Name', 'Team', 'League', 'Date', 'DATE', 'OPP', 'RESULT', 'IP', 'H',
       'R', 'ER', 'HR', 'BB', 'K', 'GB', 'FB', 'P', 'TBF', 'GSC', 'DEC', 'REL',
       'ERA', 'AB', '2B', '3B', 'RBI', 'HBP', 'SO', 'SB', 'CS', 'AVG', 'OBP',
       'SLG', 'OPS'],
      dtype='object')

In [12]:
df.head()

,Name,Team,League,Date,DATE,OPP,RESULT,IP,H,R,...,3B,RBI,HBP,SO,SB,CS,AVG,OBP,SLG,OPS
0,Tyler Anderson,LAA,MLB,2023-06-06,Tue 6/6,vs\nCHC,W\n7-4,5.0,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thairo Estrada,SFG,MLB,2023-06-10,Sat 6/10,vs\nCHC,L\n4-0,NaN,0,0,...,0,0,0,2,0,0,.293,.339,.470,.809


In [13]:
df.tail()

,Name,Team,League,Date,DATE,OPP,RESULT,IP,H,R,...,3B,RBI,HBP,SO,SB,CS,AVG,OBP,SLG,OPS
0,Tyler Anderson,LAA,MLB,2023-06-06,Tue 6/6,vs\nCHC,W\n7-4,5.0,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Thairo Estrada,SFG,MLB,2023-06-10,Sat 6/10,vs\nCHC,L\n4-0,NaN,0,0,...,0,0,0,2,0,0,.293,.339,.470,.809


NameError: name 'pd' is not defined